In [1]:
from selenium import webdriver
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep, time
import re, csv
from random import uniform, randint
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# 
from seleniumbase import SB
# import undetected_chromedriver as uc

import undetected_chromedriver as uc


In [46]:
base_url = "https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p{}?rs=2,3,4"
url_list = []
start_page = 21
end_page = 30   #20
for page in range(start_page, end_page +1):
    url = base_url.format(page)
    url_list.append(url)

In [47]:
url_list

['https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p13?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p14?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p15?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p16?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p17?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p18?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p19?rs=2,3,4',
 'https://batdongsan.com.vn/ban-nha-rieng-ha-noi/1pn/p20?rs=2,3,4']

# Function to get information

In [48]:
def get_info(link):
    driver.get(link)
    sleep(random.randint(5,10))
    while True:
#     Get links and title
        elems = driver.find_elements(By.CSS_SELECTOR, "div.js__card.js__card-full-web.pr-container.re__card-full.re__vip-normal [href]")
        title = [elem.text for elem in elems]
        links = [elem.get_attribute('href') for elem in elems]
#         link_demo = links
    #     Get price
        elems_price = driver.find_elements(By.CSS_SELECTOR, ".re__card-config-price.js__card-config-item")
        length = len(elems_price)
        price = [elem_price.text for elem_price in elems_price]

    #     Get location
        elems_loc = driver.find_elements(By.CSS_SELECTOR, ".re__card-location")
        # length_loc = len(elems_size)
        location = [elem_loc.text for elem_loc in elems_loc]

    #  Get size
        elems_size = driver.find_elements(By.CSS_SELECTOR, ".re__card-config-area.js__card-config-item")
        length_size = len(elems_size)
        size = [elem_size.text for elem_size in elems_size]

    #     Get percent price
    # <span class="re__card-config-price_per_m2 js__card-config-item">100 tr/m²</span>
        elems_per = driver.find_elements(By.CSS_SELECTOR, ".re__card-config-price_per_m2.js__card-config-item")
        length_per = len(elems_per)
        percent = [elem_per.text for elem_per in elems_per]

        df1 = pd.DataFrame(list(zip(title, price, links, location, size, percent)), columns = ['title', 'price','link_item', 'location', 'size', 'Price_per_m2'])
        df1['index_']= np.arange(1, len(df1) + 1)

    # Get bedroom
        bed_list, bed_idx = [], []
        for i in range(1, len(title)+1):
            try:
                bed = driver.find_element("xpath","/html/body/div[8]/div[2]/div[1]/div[6]/div[{}]/a/div[2]/div[1]/div[1]/div[1]/span[7]".format(i))
                bed_list.append(bed.text)
                bed_idx.append(i)
            except NoSuchElementException:
                pass

        df2 = pd.DataFrame(list(zip(bed_idx , bed_list)), columns = ['bed_idx', 'bedroom',])
        df3 = df1.merge(df2, how='left', left_on='index_', right_on='bed_idx')
    #     Get toilet
        toi_list, toi_idx = [], []
        for i in range(1, len(title)+1):
            try:
                toilet = driver.find_element("xpath","/html/body/div[8]/div[2]/div[1]/div[6]/div[{}]/a/div[2]/div[1]/div[1]/div[1]/span[9]".format(i))
                toi_list.append(toilet.text)
                toi_idx.append(i)
            except NoSuchElementException:
                pass
        df4 = pd.DataFrame(list(zip(toi_idx , toi_list)), columns = ['toi_idx', 'toi_list'])
        df5 = df3.merge(df4, how='left', left_on='index_', right_on='toi_idx')
        driver.close()
        return df5
    

all_data = []
for link in url_list:
    options = uc.ChromeOptions() 
    options.headless = False  
    driver = uc.Chrome(use_subprocess=True, options=options) 
    driver.get(link) 
    driver.maximize_window() 
    df = get_info(link)
    all_data.append(df)
    final_df = pd.concat(all_data, ignore_index=True )
    sleep(3)
    
# driver.close()

In [49]:
final_df

,title,price,link_item,location,size,Price_per_m2,index_,bed_idx,bedroom,toi_idx,toi_list
0,"1\nBán nhà Thạch Bàn, ô tô tránh trước cửa nhà...","4,3 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Long Biên, Hà Nội",35 m²,"122,86 tr/m²",1,1.0,3,1.0,4
1,"5\nBán nhà Khương Đình, ngõ thoáng, lô góc, 50...","5,08 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Thanh Xuân, Hà Nội",50 m²,"101,6 tr/m²",2,2.0,4,2.0,4
2,"8\nBán nhà Nguyễn Trãi, ngõ gần ô tô, 46m, 4 t...","4,85 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Thanh Xuân, Hà Nội",46 m²,"105,44 tr/m²",3,3.0,4,3.0,4
3,3\nRẻ nhất tại Mậu Lương ô tô đỗ cửa 64m2 nhỉn...,"4,8 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Hà Đông, Hà Nội",64 m²,75 tr/m²,4,4.0,4,4.0,3
4,"1\nBán nhà Thạch Bàn, 52m2, 5 tầng, lô góc, nở...","5,9 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Long Biên, Hà Nội",52 m²,"113,46 tr/m²",5,5.0,4,5.0,4
...,...,...,...,...,...,...,...,...,...,...,...
146,"2\nChính chủ bán nhà gần Yên Nghĩa, ô tô đỗ gầ...","1,68 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Hà Đông, Hà Nội",52 m²,"32,31 tr/m²",16,16.0,4,16.0,3
147,"Nhỉnh 1.8 tỷ có nhà đường Hòa Bình, Yên Nghĩa,...","1,85 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Hà Đông, Hà Nội",35 m²,"52,86 tr/m²",17,17.0,2,17.0,1
148,"3\nChính chủ bán nhà Xuân Diệu, Tây Hồ, 46.6m2...","8,6 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Tây Hồ, Hà Nội","46,6 m²","184,55 tr/m²",18,18.0,4,18.0,4
149,"6\nChính Chủ bán nhà 229B/22 Phúc Đồng, Long B...",3 tỷ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Long Biên, Hà Nội",31 m²,"96,77 tr/m²",19,19.0,4,19.0,4


In [50]:
meo = []
meo = final_df['link_item']


In [44]:
#  example

In [10]:
meo1 = meo[1:5]
meo1

1    https://batdongsan.com.vn/ban-nha-rieng-duong-...
2    https://batdongsan.com.vn/ban-nha-rieng-pho-xa...
3    https://batdongsan.com.vn/ban-nha-rieng-xa-phu...
4    https://batdongsan.com.vn/ban-nha-rieng-duong-...
Name: link_item, dtype: object

In [51]:
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from time import sleep

def getDetailItems(link):
    # driver = webdriver.Chrome(executable_path='/path/to/chromedriver')  # Replace with the path to your ChromeDriver executable
    driver.get(link)
    # path, floor, duong_vao = [], [], []

    # # Get mặt tiền
    # elems_path = driver.find_elements(By.CSS_SELECTOR, ".re__pr-specs-content js__other-info.re__pr-specs-content-item.re__pr-specs-content-item-value")
    # path = [elem_path.text for elem_path in elems_path]

    # # Get số tầng
    # elems_floor = driver.find_elements(By.CSS_SELECTOR, ".re__pr-specs-content js__other-info.re__pr-specs-content-item.re__pr-specs-content-item-value")
    # floor = [elem_floor.text for elem_floor in elems_floor]

    # # Get đường vào
    # elems_duong = driver.find_elements(By.CSS_SELECTOR, ".re__pr-specs-content js__other-info.re__pr-specs-content-item.re__pr-specs-content-item-value")
    # duong_vao = [elem_duong.text for elem_duong in elems_duong]

    # df0 = pd.DataFrame(list(zip(path, floor, duong_vao)), columns=['mat_tien', 'floor', 'duong_vao'])
    # df0.insert(0, "link_item", link)

    # driver.close()
    # return df0
    data = {
        "link_item": link,
        "mat_tien": None,
        "floor": None,
        "duong_vao": None,
        "phap_ly" :None,
        "noi_that" : None,
        "Huong" :None,
        "Huong_ban_cong": None,
    }

    # Find all elements with class "re__pr-specs-content-item"
    elems = driver.find_elements(By.CSS_SELECTOR, ".re__pr-specs-content-item")

    for elem in elems:
        title = elem.find_element(By.CLASS_NAME, "re__pr-specs-content-item-title").text
        value = elem.find_element(By.CLASS_NAME, "re__pr-specs-content-item-value").text

        if title == "Mặt tiền":
            data["mat_tien"] = value
        elif title == "Số tầng":
            data["floor"] = value
        elif title == "Đường vào":
            data["duong_vao"] = value
        elif title == "Pháp lý":
            data["phap_ly"] = value
        elif title == "Nội thất":
            data["noi_that"] = value
        elif title == "Hướng nhà":
            data["Huong"] = value
        elif title == "Hướng ban công":
            data["Huong_ban_cong"] = value

    df0 = pd.DataFrame([data])
    driver.close()
    return df0

# meo1 = ["url1", "url2"]  # Replace with your list of URLs
df_list = []

for link in meo:
    options = uc.ChromeOptions() 
    options.headless = False  # Set headless to False to run in non-headless mode

    driver = uc.Chrome(use_subprocess=True, options=options) 
    driver.get(link) 
    driver.maximize_window() 

    # df = getDetailItems(link)
    # df_list.append(df)
    # final_meo = pd.concat(df_list, ignore_index=True )
    # sleep(3)
    df = getDetailItems(link)
    df_list.append(df)
    sleep(3)

final_meo = pd.concat(df_list, ignore_index=True)

# Save the final DataFrame to a CSV file
# final_meo.to_csv('output.csv', index=False)

In [52]:
final_meo

,link_item,mat_tien,floor,duong_vao,phap_ly,noi_that,Huong,Huong_ban_cong
0,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,5 tầng,None,None,None,None,None
1,https://batdongsan.com.vn/ban-nha-rieng-duong-...,4 m,4 tầng,4 m,Sổ đỏ lâu dài,Nội thất liền tườnh,None,None
2,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,68 m",4 tầng,4 m,Sổ đỏ lâu dài,Nội thất liền tường,None,None
3,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,5 m",4 tầng,4 m,Sổ đỏ/ Sổ hồng,Cơ bản,Tây,None
4,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,5 tầng,None,None,None,None,None
...,...,...,...,...,...,...,...,...
146,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,2 tầng,None,Sổ đỏ,None,None,None
147,https://batdongsan.com.vn/ban-nha-rieng-duong-...,4 m,4 tầng,"2,5 m",Sổ đỏ/ Sổ hồng,Cơ bản,Tây - Nam,Tây - Nam
148,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,65 m",4 tầng,2 m,Sổ đỏ/ Sổ hồng,None,None,None
149,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"3,8 m",5 tầng,"2,5 m",Sổ đỏ/ Sổ hồng,Đầy đủ,None,None


In [53]:
concatenated_df = pd.concat([final_df, final_meo], axis=1)

# Make sure the indices align or reset them if needed
final_df.reset_index(drop=True, inplace=True)
final_meo.reset_index(drop=True, inplace=True)

# Then, concatenate horizontally
concatenated_df = pd.concat([final_df, final_meo], axis=1)

In [54]:
concatenated_df

,title,price,link_item,location,size,Price_per_m2,index_,bed_idx,bedroom,toi_idx,toi_list,link_item,mat_tien,floor,duong_vao,phap_ly,noi_that,Huong,Huong_ban_cong
0,"1\nBán nhà Thạch Bàn, ô tô tránh trước cửa nhà...","4,3 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Long Biên, Hà Nội",35 m²,"122,86 tr/m²",1,1.0,3,1.0,4,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,5 tầng,None,None,None,None,None
1,"5\nBán nhà Khương Đình, ngõ thoáng, lô góc, 50...","5,08 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Thanh Xuân, Hà Nội",50 m²,"101,6 tr/m²",2,2.0,4,2.0,4,https://batdongsan.com.vn/ban-nha-rieng-duong-...,4 m,4 tầng,4 m,Sổ đỏ lâu dài,Nội thất liền tườnh,None,None
2,"8\nBán nhà Nguyễn Trãi, ngõ gần ô tô, 46m, 4 t...","4,85 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Thanh Xuân, Hà Nội",46 m²,"105,44 tr/m²",3,3.0,4,3.0,4,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,68 m",4 tầng,4 m,Sổ đỏ lâu dài,Nội thất liền tường,None,None
3,3\nRẻ nhất tại Mậu Lương ô tô đỗ cửa 64m2 nhỉn...,"4,8 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Hà Đông, Hà Nội",64 m²,75 tr/m²,4,4.0,4,4.0,3,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,5 m",4 tầng,4 m,Sổ đỏ/ Sổ hồng,Cơ bản,Tây,None
4,"1\nBán nhà Thạch Bàn, 52m2, 5 tầng, lô góc, nở...","5,9 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Long Biên, Hà Nội",52 m²,"113,46 tr/m²",5,5.0,4,5.0,4,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,5 tầng,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,"2\nChính chủ bán nhà gần Yên Nghĩa, ô tô đỗ gầ...","1,68 tỷ",https://batdongsan.com.vn/ban-nha-rieng-phuong...,"Hà Đông, Hà Nội",52 m²,"32,31 tr/m²",16,16.0,4,16.0,3,https://batdongsan.com.vn/ban-nha-rieng-phuong...,None,2 tầng,None,Sổ đỏ,None,None,None
147,"Nhỉnh 1.8 tỷ có nhà đường Hòa Bình, Yên Nghĩa,...","1,85 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Hà Đông, Hà Nội",35 m²,"52,86 tr/m²",17,17.0,2,17.0,1,https://batdongsan.com.vn/ban-nha-rieng-duong-...,4 m,4 tầng,"2,5 m",Sổ đỏ/ Sổ hồng,Cơ bản,Tây - Nam,Tây - Nam
148,"3\nChính chủ bán nhà Xuân Diệu, Tây Hồ, 46.6m2...","8,6 tỷ",https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Tây Hồ, Hà Nội","46,6 m²","184,55 tr/m²",18,18.0,4,18.0,4,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"4,65 m",4 tầng,2 m,Sổ đỏ/ Sổ hồng,None,None,None
149,"6\nChính Chủ bán nhà 229B/22 Phúc Đồng, Long B...",3 tỷ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"Long Biên, Hà Nội",31 m²,"96,77 tr/m²",19,19.0,4,19.0,4,https://batdongsan.com.vn/ban-nha-rieng-duong-...,"3,8 m",5 tầng,"2,5 m",Sổ đỏ/ Sổ hồng,Đầy đủ,None,None


In [55]:
concatenated_df.to_csv('house4.csv')